## Artificial Neural Network

#### Importing the Libraries

In [2]:
import tensorflow as tf 
import numpy as np
import pandas as pd

### Part 1 - Data Preprocessing

#### Importing the dataset 

In [7]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [8]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [9]:
print(y)

[1 0 1 ... 1 1 0]


#### Encoding Categorical data

##### Label encoding the "Gender" column

In [10]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
X[:, 2] = lb.fit_transform(X[:, 2])

In [11]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


##### One Hot Encoding the "Geography" Column

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [14]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


#### Splitting the data into training set and test set 

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

#### Feature Scaling

In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [18]:
print(X_train)

[[-1.01460667 -0.5698444   1.74309049 ...  0.64259497 -1.03227043
   1.10643166]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497  0.9687384
  -0.74866447]
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.48533467]
 ...
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.41231994]
 [-1.01460667 -0.5698444   1.74309049 ...  0.64259497  0.9687384
   0.84432121]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497 -1.03227043
   0.32472465]]


### Part 2 - Building the ANN

#### Initialising the ANN

In [20]:
ann = tf.keras.models.Sequential()

#### Adding the input layer and the first hidden layer

In [22]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

#### Adding the second hidden layer

In [23]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

#### Adding the output layer

In [24]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid')) # activation = 'softmax' for non-binary classification

### Part 3 - Training the ANN

#### Compiling the ANN

In [26]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # loss = 'categorical_crossentropy' , for non-binary classification 

#### Training the ANN on the training set

In [27]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.5353 - accuracy: 0.7715
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4696 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4454 - accuracy: 0.7965
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4324 - accuracy: 0.7993
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4221 - accuracy: 0.8130
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4117 - accuracy: 0.8260
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3989 - accuracy: 0.8341
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3842 - accuracy: 0.8416
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3725 - accuracy: 0.8451
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3641 - accura

### Part 4 - Making the prediction and evaluating the model

#### Predicting the results of a single observation 

In [40]:
ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5

array([[False]])

#### Predicting the test set result

In [41]:
y_pred = ann.predict(X_test) 
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


#### Making the confusion matrix

In [42]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1513   82]
 [ 202  203]]


0.858